In [1]:
""" Using the preformated JSON pipeline, 
loop through a python script to download chunked LAS files based on a fishnet geometry file """

import geopandas as gpd
import pandas as pd
import numpy as np
import shapely.geometry
import pdal, json, requests, urllib.parse, geojson, mercantile, os, tempfile, fiona, pyproj
from tqdm import tqdm
from urllib.request import urlopen
from shapely.geometry import Polygon


#############################################################################################
# AOIName
name = "Boston"
print(name)
# Define Output Directory
outputdirectory = r"S:\GCMC\Data\LiDAR\{}LAS".format(name)

## Define Lidar Source
lidarsource = "http://usgs-lidar-public.s3.amazonaws.com/MA_CentralEastern_1_2021/ept.json"  # Boston

# ## 3DEP Lidar Footprints
# projecturl = "https://index.nationalmap.gov/arcgis/rest/services/3DEPElevationIndex/MapServer/24/query?"
# params = {"workunit": "MA_CentralEastern_1_2021"}
# projecturl = projecturl + urllib.parse.urlencode(params)
# response = requests.get(url=projecturl)
# data = response.text
# gdf_temp = gpd.read_file(data)
bounds = [[[-71.19045, 42.3799], [-71.19, 42.38069], [-71.18870, 42.38068], [-71.18901, 42.37966]]]


######--------------- Read in the Building Footprints
#########Define our area of interest, a LIDAR project (AOI)
aoi_geom = {
    "coordinates": bounds,
    "type": "Polygon",
}
aoi_shape = shapely.geometry.shape(aoi_geom)
print(aoi_shape)
minx, miny, maxx, maxy = aoi_shape.bounds
aoi_gdf = gpd.GeoSeries(aoi_shape,crs=4326)

output_fn = "example_building_footprints.geojson"

# ###############Determine which tiles intersect our AOI
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(int(mercantile.quadkey(tile)))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

# ########Step 3 - Download the building footprints for each tile that intersects our AOI and crop the results
df = pd.read_csv(
    "https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv"
)

idx = 0
combined_rows = []
geodf = gpd.GeoDataFrame(columns=["id", "geometry"], geometry="geometry", crs=4326)

with tempfile.TemporaryDirectory() as tmpdir:
    # Download the GeoJSON files for each tile that intersects the input geometry
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(shapely.geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            # geodf = pd.concat([geodf, gdf])
            # fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            # tmp_fns.append(fn)
            # if not os.path.exists(fn):
            #     gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")
        geodf = pd.concat([geodf, gdf])


### Clip Footprints to AOI
geodf = geodf.cx[aoi_gdf.total_bounds[0]:aoi_gdf.total_bounds[2],aoi_gdf.total_bounds[1]:aoi_gdf.total_bounds[3]]


geodf["properties"].apply(
    lambda x: x.update(
        {
            "minHAG": np.nan,
            "minHAG25": np.nan,
            "maxHAG": np.nan,
            "maxHAG25": np.nan,
            "meanHAG": np.nan,
            "meanHAG25": np.nan,
            "medHAG": np.nan,
            "medHAG25": np.nan,
            "stdevHAG25": np.nan,
            "q1HAG": np.nan,
            "q1HAG25": np.nan,
            "q3HAG": np.nan,
            "q3HAG25": np.nan,
            "ground": np.nan,
            "heightobs": np.nan,
            "heightobs25": np.nan,
        }
    )
)
# print(geodf)
gdf = geodf.to_crs(3857)
print(gdf)




Boston
POLYGON ((-71.19045 42.3799, -71.19 42.38069, -71.1887 42.38068, -71.18901 42.37966, -71.19045 42.3799))
The input area spans 1 tiles: [30233212]


100%|██████████| 1/1 [01:09<00:00, 69.96s/it]

         id                                           geometry     type  \
5211    NaN  POLYGON ((-7924809.789 5217997.417, -7924806.7...  Feature   
15556   NaN  POLYGON ((-7924759.458 5218136.937, -7924768.2...  Feature   
42855   NaN  POLYGON ((-7924745.685 5218039.819, -7924729.9...  Feature   
70048   NaN  POLYGON ((-7924741.542 5218080.977, -7924719.4...  Feature   
97552   NaN  POLYGON ((-7924690.104 5218044.841, -7924690.4...  Feature   
101021  NaN  POLYGON ((-7924759.825 5218087.037, -7924756.2...  Feature   
101022  NaN  POLYGON ((-7924822.743 5218026.089, -7924838.3...  Feature   
107713  NaN  POLYGON ((-7924763.176 5218147.417, -7924741.3...  Feature   
124787  NaN  POLYGON ((-7924792.394 5218050.798, -7924778.3...  Feature   
159238  NaN  POLYGON ((-7924726.803 5218120.780, -7924708.7...  Feature   
159245  NaN  POLYGON ((-7924849.640 5218075.016, -7924860.9...  Feature   
193589  NaN  POLYGON ((-7924755.729 5218055.748, -7924771.1...  Feature   
214187  NaN  POLYGON ((-7

In [2]:
gdf["innerbuffer"] = gdf.geometry.buffer(-1)
gdf["fpbuffer"] = gdf.geometry.buffer(3)
print(gdf)

         id                                           geometry     type  \
5211    NaN  POLYGON ((-7924809.789 5217997.417, -7924806.7...  Feature   
15556   NaN  POLYGON ((-7924759.458 5218136.937, -7924768.2...  Feature   
42855   NaN  POLYGON ((-7924745.685 5218039.819, -7924729.9...  Feature   
70048   NaN  POLYGON ((-7924741.542 5218080.977, -7924719.4...  Feature   
97552   NaN  POLYGON ((-7924690.104 5218044.841, -7924690.4...  Feature   
101021  NaN  POLYGON ((-7924759.825 5218087.037, -7924756.2...  Feature   
101022  NaN  POLYGON ((-7924822.743 5218026.089, -7924838.3...  Feature   
107713  NaN  POLYGON ((-7924763.176 5218147.417, -7924741.3...  Feature   
124787  NaN  POLYGON ((-7924792.394 5218050.798, -7924778.3...  Feature   
159238  NaN  POLYGON ((-7924726.803 5218120.780, -7924708.7...  Feature   
159245  NaN  POLYGON ((-7924849.640 5218075.016, -7924860.9...  Feature   
193589  NaN  POLYGON ((-7924755.729 5218055.748, -7924771.1...  Feature   
214187  NaN  POLYGON ((-7

In [3]:
#############################################################################################
# change the global options that Geopandas inherits from
pd.set_option('display.max_columns',None)


#row = gpd.GeoDataFrame(gdf.iloc[[0]],crs=3857)
# for count, row in gdf.iterfeatures():
def calcHeight(x):
    
    row = gpd.GeoDataFrame(pd.DataFrame(x).transpose(), crs=3857)
    row["fpbuffer"]=gpd.GeoSeries(row["fpbuffer"],crs = 3857)
    row["innerbuffer"]=gpd.GeoSeries(row["innerbuffer"],crs = 3857)
    
    
    
    footprintWKT = row.geometry.to_wkt()
    #print("FootprintWKT",footprintWKT)
    #print("FootprintWKT: ",str(footprintWKT))
    
    bufferbounds = ([row["fpbuffer"].total_bounds[0],row["fpbuffer"].total_bounds[2]],
                    [row["fpbuffer"].total_bounds[1],row["fpbuffer"].total_bounds[3]]
                    )

    innerbounds = ([row["innerbuffer"].total_bounds[0],row["innerbuffer"].total_bounds[2]],
                [row["innerbuffer"].total_bounds[1],row["innerbuffer"].total_bounds[3]]
                )

    #print("The buffer bounds: ", bufferbounds)


    ## Using the standard EPT LAS pipeline format, reprojecting the output to the same EPSG as the AOI
    rawLAS = json.dumps([
            {
                "type":"readers.ept",
                "filename":lidarsource,
                "bounds":str(bufferbounds)
            },
            # {
            #     "filename":outputdirectory+"\\"+name+"rawtestlasAOI_{}.las".format("a"+str(1))
            # }
            ])


    rawpipeline= pdal.Pipeline(rawLAS)
    rawpipeline.execute()
    rawarray=rawpipeline.arrays
    #print(rawarray)
    rawarray=np.ma.masked_where(
            rawarray[0][['Classification']]["Classification"]!=2,
            rawarray[0][['Z']]["Z"]).filled(np.nan)



    ## Run Pipeline on feature

    LAStiles = json.dumps([
                {
                    "type":"readers.ept",
                    "filename":lidarsource,
                    "bounds":str(bufferbounds)
                },
                {
                    "type":"filters.hag_nn",
                    "allow_extrapolation":"true",
                },
                {
                    "type":"filters.expression",
                    "expression":"(!(Classification ==2 || Classification ==3 || Classification ==4 || Classification ==5 || Classification ==7 || Classification ==18) && HeightAboveGround>1)"
                },
                {
                    "type":"filters.crop",
                    "polygon":footprintWKT.iloc[0]
                },
                # {
                #     "filename":outputdirectory+"\\"+name+"testlasAOI_{}.las".format("a"+str(count))
                # } 
                ])


    pipeline = pdal.Pipeline(LAStiles)
    pipeline.execute()
    arrays = pipeline.arrays


    ## Perform calculations on array and append to feature
    if arrays[0][['HeightAboveGround']]['HeightAboveGround'].size>0:
        top25array = np.ma.masked_where(
            arrays[0][['HeightAboveGround']]["HeightAboveGround"]<np.percentile(arrays[0][['HeightAboveGround']]["HeightAboveGround"],25),
            arrays[0][['HeightAboveGround']]["HeightAboveGround"]).filled(np.nan)
        heightarray = arrays[0][['HeightAboveGround']]["HeightAboveGround"]
        groundarray = rawarray

        #Calculate Sample Size
        heightobs = np.sum(~np.isnan(heightarray))
        heightobs25 = np.sum(~np.isnan(top25array))

        #Calculate Min/Max Values
        meanground = np.nanmean(groundarray)
        minHAG = np.nanmin(heightarray)
        min25HAG = np.nanmin(top25array)
        maxHAG = np.nanmax(heightarray)
        max25HAG = np.nanmax(top25array)

        #Calculate Mean Values
        meanHAG = np.nanmean(heightarray)
        mean25HAG = np.nanmean(top25array)
        meanZ = np.nanmean(arrays[0][['Z']]["Z"])

        #Calculate Standard Deviations
        stdHAG = np.nanstd(heightarray)
        std25HAG = np.nanstd(top25array)

        #Calculate Median Values
        medHAG = np.nanmedian(heightarray)
        med25HAG = np.nanmedian(top25array)

        #Calculate Quartile 1
        q3HAG, q1HAG = np.nanpercentile(heightarray, [75 ,25])
        q325HAG, q125HAG = np.nanpercentile(top25array, [75 ,25])


    else:
        heightobs = np.nan
        heightobs25 = np.nan
        meanground=np.nan
        minHAG= np.nan
        min25HAG= np.nan
        maxHAG= np.nan
        max25HAG= np.nan
        meanHAG= np.nan
        mean25HAG= np.nan
        medHAG= np.nan
        med25HAG= np.nan
        stdHAG= np.nan
        std25HAG= np.nan
        q1HAG= np.nan
        q125HAG= np.nan
        q3HAG= np.nan
        q325HAG = np.nan

    return {
        "minHAG":minHAG,
        "min25HAG":min25HAG,
        "maxHAG":maxHAG,
        "max25HAG":max25HAG,
        "meanHAG":meanHAG,
        "mean25HAG":mean25HAG,
        "medHAG":medHAG,
        "med25HAG":med25HAG,
        "stdHAG":stdHAG,
        "std25HAG":std25HAG,
        "q1HAG":q1HAG,
        "q125HAG":q125HAG,
        "q3HAG":q3HAG,
        "q325HAG":q325HAG,
        "meanground":meanground,
        "heightobs":heightobs,
        "heightobs25":heightobs25
        }
    
    



In [4]:
print(gdf)
gdfout=gdf.copy()


#results = gdfout.apply(lambda ro: calcHeight(ro),axis=1)



         id                                           geometry     type  \
5211    NaN  POLYGON ((-7924809.789 5217997.417, -7924806.7...  Feature   
15556   NaN  POLYGON ((-7924759.458 5218136.937, -7924768.2...  Feature   
42855   NaN  POLYGON ((-7924745.685 5218039.819, -7924729.9...  Feature   
70048   NaN  POLYGON ((-7924741.542 5218080.977, -7924719.4...  Feature   
97552   NaN  POLYGON ((-7924690.104 5218044.841, -7924690.4...  Feature   
101021  NaN  POLYGON ((-7924759.825 5218087.037, -7924756.2...  Feature   
101022  NaN  POLYGON ((-7924822.743 5218026.089, -7924838.3...  Feature   
107713  NaN  POLYGON ((-7924763.176 5218147.417, -7924741.3...  Feature   
124787  NaN  POLYGON ((-7924792.394 5218050.798, -7924778.3...  Feature   
159238  NaN  POLYGON ((-7924726.803 5218120.780, -7924708.7...  Feature   
159245  NaN  POLYGON ((-7924849.640 5218075.016, -7924860.9...  Feature   
193589  NaN  POLYGON ((-7924755.729 5218055.748, -7924771.1...  Feature   
214187  NaN  POLYGON ((-7

In [5]:
gdfout.apply(lambda x: x["properties"].update(calcHeight(x)),axis=1)
gdfout.reset_index(inplace=True)
gdfout = gdfout.join(pd.json_normalize(gdfout.pop('properties')))
gdfout=gdfout.drop(columns=["innerbuffer","fpbuffer"])



,index,id,geometry,type,height,minHAG,minHAG25,maxHAG,maxHAG25,meanHAG,meanHAG25,medHAG,medHAG25,stdevHAG25,q1HAG,q1HAG25,q3HAG,q3HAG25,ground,heightobs,heightobs25,min25HAG,max25HAG,mean25HAG,med25HAG,stdHAG,std25HAG,q125HAG,q325HAG,meanground
0,5211,NaN,"POLYGON ((-7924809.789 5217997.417, -7924806.7...",Feature,5.769141,1.01,NaN,9.18,NaN,6.913686,NaN,7.090,NaN,NaN,6.5900,NaN,7.7100,NaN,NaN,1674,1256,6.59,9.18,7.468822,7.360,1.412225,0.596534,6.9900,7.8900,29.757545
1,15556,NaN,"POLYGON ((-7924759.458 5218136.937, -7924768.2...",Feature,-1.000000,1.69,NaN,4.52,NaN,3.092908,NaN,3.075,NaN,NaN,2.7100,NaN,3.4500,NaN,NaN,368,277,2.71,4.52,3.314874,3.280,0.541487,0.400860,2.9800,3.5700,28.814919
2,42855,NaN,"POLYGON ((-7924745.685 5218039.819, -7924729.9...",Feature,4.809786,1.08,NaN,11.48,NaN,7.956024,NaN,8.460,NaN,NaN,7.3300,NaN,9.6500,NaN,NaN,1064,799,7.33,11.48,9.035181,9.110,2.343425,1.008308,8.1600,9.8600,30.273967
3,70048,NaN,"POLYGON ((-7924741.542 5218080.977, -7924719.4...",Feature,5.012177,1.04,NaN,8.55,NaN,7.018938,NaN,7.400,NaN,NaN,6.7600,NaN,7.8700,NaN,NaN,1413,1061,6.76,8.55,7.627342,7.630,1.485513,0.470337,7.2700,8.0000,29.283318
4,97552,NaN,"POLYGON ((-7924690.104 5218044.841, -7924690.4...",Feature,6.792814,1.41,NaN,11.76,NaN,8.806982,NaN,9.970,NaN,NaN,8.2600,NaN,10.6500,NaN,NaN,888,666,8.27,11.76,10.259520,10.310,2.705701,0.722727,9.7900,10.8275,31.939822
5,101021,NaN,"POLYGON ((-7924759.825 5218087.037, -7924756.2...",Feature,5.503683,1.01,NaN,10.32,NaN,8.549726,NaN,9.095,NaN,NaN,7.7350,NaN,9.6400,NaN,NaN,2154,1615,7.75,10.32,9.290978,9.370,1.572145,0.569208,8.9500,9.7400,29.211849
6,101022,NaN,"POLYGON ((-7924822.743 5218026.089, -7924838.3...",Feature,4.946511,1.03,NaN,6.94,NaN,5.198083,NaN,5.600,NaN,NaN,4.5950,NaN,6.0200,NaN,NaN,819,614,4.60,6.94,5.702769,5.860,1.083484,0.536217,5.2425,6.1000,29.719846
7,107713,NaN,"POLYGON ((-7924763.176 5218147.417, -7924741.3...",Feature,4.738742,1.57,NaN,10.15,NaN,6.999312,NaN,7.700,NaN,NaN,4.5000,NaN,8.7000,NaN,NaN,1191,894,4.50,10.15,7.999787,8.250,2.001566,1.138691,7.3000,8.9400,28.883518
8,124787,NaN,"POLYGON ((-7924792.394 5218050.798, -7924778.3...",Feature,4.396695,1.02,NaN,8.99,NaN,6.427325,NaN,6.805,NaN,NaN,6.1900,NaN,7.4800,NaN,NaN,658,494,6.19,8.99,7.156579,7.140,1.555858,0.559418,6.6725,7.6700,29.061178
9,159238,NaN,"POLYGON ((-7924726.803 5218120.780, -7924708.7...",Feature,4.208093,1.01,NaN,10.85,NaN,6.308753,NaN,5.330,NaN,NaN,4.0250,NaN,8.5700,NaN,NaN,1091,818,4.03,10.85,7.217543,7.440,2.483841,2.204530,4.8900,9.1575,29.132481


In [11]:
#gdfout=gdfout.drop(columns=["innerbuffer","fpbuffer"])
print(gdfout.columns)
print(gdfout.dtypes)
gdfout.to_file(r"S:\GCMC\_Code\temp\building.shp")

Index(['index', 'id', 'geometry', 'type', 'height', 'minHAG', 'minHAG25',
       'maxHAG', 'maxHAG25', 'meanHAG', 'meanHAG25', 'medHAG', 'medHAG25',
       'stdevHAG25', 'q1HAG', 'q1HAG25', 'q3HAG', 'q3HAG25', 'ground',
       'heightobs', 'heightobs25', 'min25HAG', 'max25HAG', 'mean25HAG',
       'med25HAG', 'stdHAG', 'std25HAG', 'q125HAG', 'q325HAG', 'meanground'],
      dtype='object')
index             int64
id               object
geometry       geometry
type             object
height          float64
minHAG          float64
minHAG25        float64
maxHAG          float64
maxHAG25        float64
meanHAG         float64
meanHAG25       float64
medHAG          float64
medHAG25        float64
stdevHAG25      float64
q1HAG           float64
q1HAG25         float64
q3HAG           float64
q3HAG25         float64
ground          float64
heightobs         int32
heightobs25       int32
min25HAG        float64
max25HAG        float64
mean25HAG       float64
med25HAG        float64
stdHAG  

C:\Users\wik191\AppData\Local\Temp\ipykernel_24528\4186084088.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdfout.to_file(r"S:\GCMC\_Code\temp\building.shp")
